In [19]:
#import the libraries
import pandas as pd
import numpy as np
import yfinance as yf
from statsmodels.tsa.api import VAR

In [20]:
# Scrapping data and creation of dataset
wti_oil = yf.download(tickers="CL=F",period="5y",interval="1d")
wti_oil = wti_oil.drop(columns=['Open','High','Low','Adj Close','Volume'],axis=1)

#FEATURE 1 - UTILIZATION OF REFINERY
feature_1 = pd.read_excel("C:/Users/prathikm/Downloads/project/FINAL_FILES/to_submit/feature_datasets/" + "US Refinery Utilization Rates.xls", sheet_name="Data 1")
feature_1 = feature_1.drop([0,1])
feature_1= feature_1.rename(columns={"Back to Contents":"Date", "Data 1: Weekly U.S. Percent Utilization of Refinery Operable Capacity (Percent)":"Utilization of Refinery in %"})
feature_1['Date'] = pd.to_datetime(feature_1['Date'])
feature_1_inx_date = feature_1.set_index(['Date'])
feature_1_inx_date_filter = feature_1_inx_date.loc['2016-12-27':'2021-12-27']  #**** PUT TODAY'S DATE & MONTH IN BOTH FIELDS, ONLY YEAR CHANGES****
post_f1 = pd.concat([wti_oil,feature_1_inx_date_filter], axis=1, join='outer')

#FEATURE 2 - LAST DAY'S CLOSE PRICE
post_f1['Last Day\'s Close'] = post_f1.Close.shift(1,fill_value=53.900) #****PUT FIRST VALUE OF CLOSE*****

#FEATURE 3 AND 4 - PRIDUCTION AND DEMAND OF OIL
feature_345 = pd.read_csv("C:/Users/prathikm/Downloads/project/FINAL_FILES/to_submit/feature_datasets/" + "World Oil Demand and Supply.csv", sep=";")
feature_345 = feature_345.drop([0,1,2])
feature_345 = feature_345[['Source:','IEA.','All']]
feature_345 = feature_345.tail(5)
feature_345 = feature_345.reset_index()
feature_345 = feature_345.drop(columns=['index'],axis=1)
feature_345= feature_345.rename(columns = {'Source:':'Year','IEA.':'Production','All':'Demand'})
feature_345.loc[len(feature_345.index)] = [2021, 4618.37,4442.05] #Change 1: 2021 will later get a fixed value, 2022 can then be calculated 
fi = feature_345['Year'].astype(int)
feature_345['Year'] = fi.values
feature_345 = feature_345.replace({'Year': {2016: '2016-12-27', 2017: '2017-01-01', 2018:'2018-01-01' ,2019:'2019-01-01' ,2020:'2020-01-01' ,2021:'2021-12-26'}}) 
                                            #****** SET 2016: TODAY's date, 2017 to 2020: 1st of Jan of that year, 2021: yesterday's date)******
feature_345 = feature_345.rename(columns={'Year':'Date'})
feature_345['Date'] = pd.to_datetime(feature_345['Date'])
feature_345 = feature_345.set_index(['Date'])
posz = pd.concat([post_f1,feature_345], axis=1, join='outer')

#FEATURE 5 - NYMEX FUTURE PRICE CONTRACT 1
feature_8_1 = pd.read_excel("C:/Users/prathikm/Downloads/project/FINAL_FILES/to_submit/feature_datasets/" + "First Contract.xls", sheet_name="Data 1")
feature_8_1 = feature_8_1.drop([0,1])
feature_8_1= feature_8_1.rename(columns={"Back to Contents":"Date", "Data 1: Cushing, OK Crude Oil Future Contract 1 (Dollars per Barrel)":"Contract 1 FP"})
feature_8_1['Date'] = pd.to_datetime(feature_8_1['Date'])
feature_8_n = feature_8_1[(feature_8_1['Date'] > '2016-12-25') & (feature_8_1['Date'] < '2021-12-27')] #****** > Today's date 2016 year - 2 days, & < today's date
feature_8_n2 = feature_8_n.set_index(['Date'])
post_contract1 = pd.concat([posz,feature_8_n2], axis=1, join='outer')

#FEATURE 6 - ELECTRONIC VEHICLE GLOBAL SALES
col_names = ["Dates","China","USA","4","5","6","7","Global Sale (in millions)"]
feature_9 = pd.read_csv("C:/Users/prathikm/Downloads/project/FINAL_FILES/to_submit/feature_datasets/" + "Global EV Sales.csv", names=col_names,error_bad_lines=False)
feature_9= feature_9.drop([0,1,2,3])
feature_9 = feature_9.drop(columns=['China','USA','4','5','6','7'], axis =1)
feature_9 = feature_9.tail(5)
feature_9 = pd.DataFrame(feature_9)
feature_9.loc[len(feature_9.index)] = [2021, 5.6] #Change 2: 2022 can have different ev sales value
dates_int = feature_9['Dates'].astype(int)
gs_int = feature_9['Global Sale (in millions)'].astype(float)
feature_9['Dates'] = dates_int.values
feature_9['Global Sale (in millions)'] = gs_int.values
feature_9 = feature_9.reset_index()
feature_9.drop(columns = {'index'},inplace = True)
feature_9 = feature_9.replace({'Dates': {2016: '2016-12-27', 2017: '2017-01-01',2018:'2018-01-01' ,2019:'2019-01-01' ,2020:'2020-01-01' ,2021:'2021-01-01'}}) 
                                                    #********* 2016: TODAY's date *******
feature_9 = feature_9.rename(columns={'Dates':'Date'})
feature_9['Date'] = pd.to_datetime(feature_9['Date'])
feature_9 = feature_9.set_index(['Date'])
post_f9 = pd.concat([post_contract1,feature_9], axis=1, join='outer')
post_f7_no_inx = post_f9.reset_index()

post_f200_inx_date = post_f7_no_inx.set_index('Date')

#CONVERT VALUES TO FLOAT
cols = ['Utilization of Refinery in %', 'Production', 'Demand','Contract 1 FP']
post_f200_inx_date[cols] = post_f200_inx_date[cols].astype(float)

#LINEARLY INTERPOLATE THE NA VALUES, ALSO BACKWARD FOR FEATURE 1
post_f200_inx_date.interpolate(method='linear',inplace = True)
post_f200_inx_date['Utilization of Refinery in %'].interpolate(method='linear',inplace = True,downcast = 'infer', limit_direction = 'backward')

#RE-ARRANGE AND RENAME FEATURES
post_f200_inx_date = post_f200_inx_date[['Utilization of Refinery in %','Production','Demand','Contract 1 FP','Global Sale (in millions)','Last Day\'s Close', 'Close']]
post_f200_inx_date.rename(columns = {"Last Day's Close":"Last Day's Price", "Close":"Price"}, inplace = True)
post_f200_no_inx_date = post_f200_inx_date.reset_index()
data = post_f200_no_inx_date.copy()
data = pd.DataFrame(data)

#HANDLE COVID OUTLIER
data_drop_date = data.drop(columns=['Date'])
start = data_drop_date.iloc[:821]
covid_period = data_drop_date.iloc[821:907]
end = data_drop_date.iloc[907:993]
last = data_drop_date.iloc[993:]
complete = pd.concat([start,end,end,last])
complete = complete.reset_index()
complete = complete.drop(columns=['index'])
complete['Date'] = data['Date']
complete = complete.set_index('Date')

complete.to_csv("Dataset.csv")

#Train Test Split
df = complete.copy()
nobs = 8
df_train, df_test = df[0:-nobs], df[-nobs:]

#Differencing for stationary
df_differenced = df_train.diff().dropna()

#VAR
model = VAR(df_differenced)
x = model.select_order(maxlags=12)

#Train the VAR Model of Selected Order(p)
model_fitted = model.fit(2)

#Forecast VAR model using statsmodels / get the lag order
lag_order = model_fitted.k_ar

#Input data for forecasting
forecast_input = df_differenced.values[-lag_order:]

#Forecast
fc = model_fitted.forecast(y=forecast_input, steps=nobs)
df_forecast = pd.DataFrame(fc, index=df.index[-nobs:], columns=df.columns + '_2d')

# Invert the transformation to get the real forecast
def invert_transformation(df_train, df_forecast, second_diff=False):
    """Revert back the differencing to get the forecast to original scale."""
    df_fc = df_forecast.copy()
    columns = df_train.columns
    for col in columns:        
        # Roll back 2nd Diff
        if second_diff:
            df_fc[str(col)+'_1d'] = (df_train[col].iloc[-1]-df_train[col].iloc[-2]) + df_fc[str(col)+'_2d'].cumsum()
        # Roll back 1st Diff
        df_fc[str(col)+'_forecast'] = df_train[col].iloc[-1] + df_fc[str(col)+'_1d'].cumsum()
    return df_fc

df_results = invert_transformation(df_train, df_forecast, second_diff=True)        
df_results.loc[:, ['Utilization of Refinery in %_forecast', 'Production_forecast', 'Demand_forecast', 'Contract 1 FP_forecast',
                   'Global Sale (in millions)_forecast', 'Last Day\'s Price_forecast', 'Price_forecast']]

#Create DataFrame for prediction
test_price = df_test['Price']
test_price = pd.DataFrame(test_price)

forecast_price = df_results['Price_forecast']
forecast_price = pd.DataFrame(forecast_price)

data_VAR_P = pd.concat([test_price, forecast_price],axis=1,join='outer')
data_VAR_P

[*********************100%***********************]  1 of 1 completed


C:\Users\prathikm\miniconda3\envs\ds\lib\site-packages\IPython\core\interactiveshell.py:3444: FutureWarning: The error_bad_lines argument has been deprecated and will be removed in a future version.


  exec(code_obj, self.user_global_ns, self.user_ns)
C:\Users\prathikm\miniconda3\envs\ds\lib\site-packages\statsmodels\tsa\base\tsa_model.py:593: ValueWarning: A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.
  warnings.warn('A date index has been provided, but it has no'


,Price,Price_forecast
Date,,
2021-12-16,72.379997,70.947612
2021-12-17,70.860001,71.057675
2021-12-20,68.230003,71.175224
2021-12-21,71.120003,71.297393
2021-12-22,72.760002,71.421874
2021-12-23,73.790001,71.553611
2021-12-26,73.275002,71.689496
2021-12-27,72.760002,71.831100
